<a href="https://colab.research.google.com/github/prititaliya/Deep-Learning/blob/main/Deep_Learnin_Hyperperametetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from sklearn.metrics import r2_score
import kerastuner as kt


In [ ]:
X,y=load_diabetes(return_X_y=True,as_frame=True)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
model=Sequential()
model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='linear'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_squared_error'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,729 (38.00 KB)

 Trainable params: 9,729 (38.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history=model.fit(X_train,y_train,epochs=10,validation_split=0.2,batch_size=32)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 27158.9902 - mean_squared_error: 27158.9902 - val_loss: 30842.7891 - val_mean_squared_error: 30842.7930
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 28664.4688 - mean_squared_error: 28664.4688 - val_loss: 30785.0352 - val_mean_squared_error: 30785.0352
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 30408.5918 - mean_squared_error: 30408.5918 - val_loss: 30691.0273 - val_mean_squared_error: 30691.0273
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 27844.0996 - mean_squared_error: 27844.0996 - val_loss: 30535.4434 - val_mean_squared_error: 30535.4434
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 27338.3730 - mean_squared_error: 27338.3730 - val_loss: 30288.7363 - val_mean_squared_error: 30288.7363
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 27628.0391 - mean_squared_error: 27628.0391 - val_loss: 29916.9082 - val_mean_squared_error: 29916.9082
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━

In [ ]:
preds=model.predict(X_test)
print(r2_score(y_test,preds))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
-3.7532965900166477


# how to select optimizer

In [ ]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(1,activation='linear'))

    opt=hp.Choice('optimizer',values=['adam','sgd','rmsprop'])
    model.compile(optimizer=opt,loss='mean_squared_error',metrics=['mean_squared_error'])
    return model

In [ ]:
tuners=kt.RandomSearch(build_model,objective='val_mean_squared_error',max_trials=10,executions_per_trial=3,directory='project1',project_name='diabetes1')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuners.search(X_train,y_train,epochs=5,validation_split=0.2,batch_size=32)

Trial 3 Complete [00h 00m 05s]
val_mean_squared_error: 2524545269375689.5

Best val_mean_squared_error So Far: 29515.461588541668
Total elapsed time: 00h 00m 19s


In [ ]:
tuners.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
model=tuners.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,729 (38.00 KB)

 Trainable params: 9,729 (38.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_split=0.2,batch_size=32,initial_epoch=6)

Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 25326.8203 - mean_squared_error: 25326.8203 - val_loss: 28435.3340 - val_mean_squared_error: 28435.3340
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 25553.2812 - mean_squared_error: 25553.2812 - val_loss: 27624.0938 - val_mean_squared_error: 27624.0938
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 24575.1836 - mean_squared_error: 24575.1836 - val_loss: 26743.5508 - val_mean_squared_error: 26743.5508
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 22219.1875 - mean_squared_error: 22219.1875 - val_loss: 25760.7324 - val_mean_squared_error: 25760.7324
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 22597.9805 - mean_squared_error: 22597.9805 - val_loss: 24692.9121 - val_mean_squared_error: 24692.9121
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23206.1309 - mean_squared_error: 23206.1309 - val_loss: 23546.6680 - val_mean_squared_error: 23546.6680
Epoch 13/20
9/9 ━━━━━━━━━━━━━━━━━━━━

# how to select nums of neuron in any layers

In [ ]:
def build_model(hp):
    model=Sequential()
    units=hp.Int(
        'units',
        5,2000,
        step=10
    )

    model.add(Dense(units,activation='relu',input_dim=X_train.shape[1]))

    model.add(Dense(1,activation='linear'))

    model.compile(
        optimizer='rmsprop',
        loss='mean_squared_error',
        metrics=['mean_squared_error']
    )

    return model

In [ ]:
tuner=kt.RandomSearch(
    build_model,
    objective='val_mean_squared_error',
    max_trials=3,
    overwrite=True
)

tuner.search(X_train,y_train,epochs=5,validation_split=0.2,batch_size=32)

Trial 3 Complete [00h 00m 02s]
val_mean_squared_error: 30288.044921875

Best val_mean_squared_error So Far: 29630.35546875
Total elapsed time: 00h 00m 07s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 1145}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,epochs=20,validation_split=0.2,batch_size=32,initial_epoch=5)

Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 27289.1094 - mean_squared_error: 27289.1094 - val_loss: 29040.8848 - val_mean_squared_error: 29040.8848
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 25039.4531 - mean_squared_error: 25039.4531 - val_loss: 28589.6582 - val_mean_squared_error: 28589.6582
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 25655.1309 - mean_squared_error: 25655.1309 - val_loss: 28135.4512 - val_mean_squared_error: 28135.4512
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 24608.3027 - mean_squared_error: 24608.3027 - val_loss: 27656.3262 - val_mean_squared_error: 27656.3262
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 24801.2324 - mean_squared_error: 24801.2324 - val_loss: 27158.7988 - val_mean_squared_error: 27158.7988
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 24166.9199 - mean_squared_error: 24166.9199 - val_loss: 26638.0859 - val_mean_squared_error: 26638.0859
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
print(r2_score(y_test,model.predict(X_test)))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
-2.720504951148903


# how to select nums of layers

In [ ]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(1145,activation='relu',input_dim=X_train.shape[1]))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(1145,activation='relu'))

    model.add(Dense(1,activation='linear'))

    model.compile(
        optimizer='rmsprop',
        loss='mean_squared_error',
        metrics=['mean_squared_error']
    )

    return model

In [ ]:
tuner=kt.RandomSearch(
    build_model,
    objective='val_mean_squared_error',
    max_trials=3,
    overwrite=True
)

tuner.search(X_train,y_train,epochs=5,validation_split=0.2,batch_size=32)

Trial 3 Complete [00h 00m 11s]
val_mean_squared_error: 3630.36669921875

Best val_mean_squared_error So Far: 3115.7578125
Total elapsed time: 00h 00m 26s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=20,validation_split=0.2,batch_size=32,initial_epoch=5)
print(r2_score(y_test,model.predict(X_test)))

Epoch 6/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 76591.8516 - mean_squared_error: 76591.8516 - val_loss: 3365.0391 - val_mean_squared_error: 3365.0391
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 2589.2686 - mean_squared_error: 2589.2686 - val_loss: 3216.4065 - val_mean_squared_error: 3216.4065
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 2407.4797 - mean_squared_error: 2407.4797 - val_loss: 3229.6575 - val_mean_squared_error: 3229.6575
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 2857.0474 - mean_squared_error: 2857.0474 - val_loss: 3081.7249 - val_mean_squared_error: 3081.7249
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 2776.5349 - mean_squared_error: 2776.5349 - val_loss: 3152.8467 - val_mean_squared_error: 3152.8467
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 2916.8247 - mean_squared_error: 2916.8247 - val_loss: 3535.2935 - val_mean_squared_error: 3535.2935
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 3145.00

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
-0.003512030360422802


# final model

In [63]:
def final_build_model(hp):
    model=Sequential()

    counter=0;

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Dense(
                hp.Int('units'+str(i),min_value=1,max_value=1500,step=np.random.
                randint(1,10)),
                activation=hp.Choice('activation'+str(i),values=['relu','tanh','softmax'])
                ,input_dim=X_train.shape[1]
            ))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5])))
            model.add(BatchNormalization())
        else:
            model.add(Dense(
                hp.Int('units'+str(i),min_value=1,max_values=1500,step=
                       np.random.randint(1,10)),
                activation=hp.Choice('activation'+str(i),values=['relu','tanh','softmax'])
            ))
            model.add(Dropout(
                hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5])
            ))
            model.add(BatchNormalization())
        counter+=1

    model.add(Dense(1,activation='linear'))

    model.compile(
        optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop']),
        loss='mean_squared_error',
        metrics=['mean_squared_error']
    )

    return model


In [64]:
final_tuner=kt.RandomSearch(
    final_build_model,
    objective='val_mean_squared_error',
    max_trials=3,
    overwrite=True
)
final_tuner.search(X_train,y_train,epochs=5,validation_split=0.2,batch_size=32)
final_model=final_tuner.get_best_models(num_models=1)[0]
final_model.fit(X_train,y_train,epochs=20,validation_split=0.2,batch_size=32,initial_epoch=5)
print(r2_score(y_test,final_model.predict(X_test)))

Trial 3 Complete [00h 00m 01s]

Best val_mean_squared_error So Far: 28579.76171875
Total elapsed time: 00h 00m 05s
Epoch 6/20


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 22231.9199 - mean_squared_error: 22231.9199 - val_loss: 27376.8691 - val_mean_squared_error: 27376.8691
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 22906.8281 - mean_squared_error: 22906.8281 - val_loss: 26558.0801 - val_mean_squared_error: 26558.0801
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 21344.4023 - mean_squared_error: 21344.4023 - val_loss: 25761.2676 - val_mean_squared_error: 25761.2676
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 20949.1016 - mean_squared_error: 20949.1016 - val_loss: 24920.1367 - val_mean_squared_error: 24920.1367
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 18572.1152 - mean_squared_error: 18572.1152 - val_loss: 24025.4473 - val_mean_squared_error: 24025.4473
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 17848.8105 - mean_squared_error: 17848.8105 - val_loss: 23111.8359 - val_mean_squared_error: 23111.8359
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms

In [67]:
final_tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 1396,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'rmsprop'}